# Stephen Lennox

## Research question/interests

I am very interested in the current housing markets in Canada. I am currently enrolled in the co-op program and due to a general lack of success on my remote/local job hunt, I am forced to look outside of the province at on-site positions to fufill my co-op graduation requirements. That means that in other provinces like Alberta, Quebec and Ontario: I need to know what the status of housing is and where in order to make informed financial choices on where I potentially move to. My research question is: *What is the best place to live in provincial capital cities based the housing index of the last year?*

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd

In [5]:
df_index= pd.read_csv("../data/raw/Montly Data/Montly-index.csv")
df_quality = pd.read_csv("../data/raw/Quality Long-Form Data/Housing-quality.csv")